<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1: VGG16 Classifier Pre-Trained Model</a>
3. <a href="#item43">Part 2: ResNET50 Classifier Pre-Trained Model 2</a>  
4. <a href="#item44">Part 3: Predict Images</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
#import shutil

#shutil.rmtree('__MACOSX')

In [2]:
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [3]:
#!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1: VGG16 Classifier Pre-Trained Model

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input



In [3]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [4]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [ ]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

In [9]:
#get validation images
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [10]:
#Build, Compile and Fit the Model 
model_vgg16 = Sequential()

In [11]:
#add the VGG16 Pre-Trained Model 
model_vgg16.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 1s 0us/step


In [12]:
#define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function
model_vgg16.add(Dense(num_classes, activation='relu'))

In [13]:
#access the layers
model_vgg16.layers

In [14]:
#access the VGG16 layer 
model_vgg16.layers[0].layers

In [15]:
#training dense layer only as VGG16 is pre-trained
model_vgg16.layers[0].trainable = False

In [16]:
#how many parameters we will need to optimize in order to train the output layer
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
#compile the model using the adam optimizer
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
#define how many steps compose an epoch
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [19]:
print("steps_per_epoch_training ", steps_per_epoch_training )
print("steps_per_epoch_validation ", steps_per_epoch_validation)

steps_per_epoch_training  301
steps_per_epoch_validation  96


In [ ]:
#fit our model
fit_history = model_vgg16.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2


In [ ]:
#save our model
model_vgg16.save('classifier_vgg16_model.h5')

In [ ]:
#plot model
#plot_model(model_vgg16, to_file='vgg16.png')

<a id="item43"></a>

## Part 2: ResNET50 Classifier Pre-Trained Model

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

In [ ]:
#load the saved ResNet50 Model
model_ResNet = load_model('classifier_resnet_model.h5')
# summarize model.
model_ResNet.summary()

In [ ]:
num_classes = 2
image_resize = 224
batch_size_testing = 100

In [ ]:
#Construct ImageDataGenerator Instances
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [ ]:
test_generator = data_generator.flow_from_directory(batch_size=batch_size_testing,
                                                    directory='concrete_data_week4/test',
                                                    shuffle=False,
                                                    target_size=(image_resize, image_resize), 
                                                    class_mode='categorical')

In [ ]:
#Performance of VGG16 Model
print("The Performance by VGG16 model is below")
VGG=model_vgg16.evaluate_generator(test_generator)
print(VGG)
print("Loss is ",str(VGG[0]))
print("The accuracy is ",str(VGG[1]))


In [ ]:
#Performance of ResNet50 Model
print("The Performance by ResNet50 model is below")
RESNET=model_ResNet.evaluate_generator(test_generator)
print(RESNET)
print("Loss is ",str(RESNET[0]))
print("The accuracy is ",str(RESNET[1]))

<a id="item44"></a>

## Part 3: Predict Images

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
vgg_pred=model_vgg16.predict_generator(test_generator)
res_pred=model_ResNet.predict_generator(test_generator)

In [ ]:
print ("VGG16 Pre-Trained Model Predict for the first 5 images in test set:")

In [ ]:
#Predict for VGG16 for the first five images in the test set
for i in vgg_pred:
    j=np.argmax(i)
    if(j==0):
      print("Negetive")
    else:
      print("Positive")

In [ ]:
print ("ResNet50 Pre-Trained Model Predict for the first 5 images in test set:")

In [ ]:
#Predict for ResNet for the first five images in the test set
for i in res_pred:
    j=np.argmax(i)
    if(j==0):
      print("Negetive")
    else:
      print("Positive")

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).